# Compilation

## "Just ahead of time" compilation

* Julia **specializes on the types of function arguments** and 
* compiles efficient machine code **when a function is called for the first time** (with these input argument types).

If the same function is called again with the same input argument types, the already existing machine code is reused.

In [ ]:
func(x,y) = 2x + y

In [ ]:
x = [1.2, 3.4, 5.6] # Vector{Float64}
y = [0.4, 0.7, 0.9] # Vector{Float64}

@time func(x,y);
@time func(x,y);

**First call:** compilation + running the code

**Second call:** running the code


If one of the input types changes, Julia compiles a new specialization of the function!


In [ ]:
typeof(x)

In [ ]:
x = [1, 3, 5]

In [ ]:
typeof(x)

In [ ]:
@time func(x,y); # Vector{Int64}, Vector{Float64}
@time func(x,y);

We now have two efficient native codes in the cache: one for all `Vector{Float64}` inputs and another one for `Vector{Int64}` as the first and `Vector{Float64}` as the second argument type.

In [ ]:
methods(func)

In [ ]:
collect(Base.specializations(only(methods(func))))

### Compilation pipeline

<p><br><img src="../images/Julia_compilation_pipeline.svg" width="512"/></p>

* **AST**: abstract syntax tree
* **IR**: intermediate representation

More about Julia compilation, see [Bezanson J et al (2018) Julia: dynamism and performance reconciled by design. Proc ACM Program Lang.](https://doi.org/10.1145/3276490)

### What makes Julia fast?

**Specialization** → (Successful) **Type inference** → **Compilation**

## Introspection tools
#### (*But I really want to see what happens!*)

We can inspect the code at all transformation stages with a bunch of macros:

<img src="../images/julia_introspection_macros.svg" width=300px>

In [ ]:
@code_lowered func(x,y)

In [ ]:
@code_typed func(x,y)

From the types of the input arguments, Julia has figured out all the intermediate types. This crucial process is known as **type inference** and its success is the basis for a good specialization (i.e. performant native code as a result). Moreover, the generic power function computing the cubic of `x` is replaced by specific floating-point multiplications (**static dispatch**).

In [ ]:
@code_llvm debuginfo=:none func(x,y)

## User Code is Fast

**In Julia, user code and user types can be just as fast as built-in or library code.**

If you know what you're doing, you can typically match the performance of optimized C/Fortran code.

In the following, we illustrate this with two basic examples.

### User code: Summation

In [ ]:
using BenchmarkTools

In [ ]:
x = rand(10^7); # some numbers to be summed up

#### C

In [ ]:
# What this does: compile simple C sum function into a shared library by piping C code into gcc

c_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
""";

using Libdl
Clib = tempname() * "." * Libdl.dlext

open(`gcc -fPIC -O3 -march=native -xc -shared -o $Clib -`, "w") do f
    print(f, c_code)
end

In [ ]:
# Readily call the function `c_sum` in the shared library

c_sum(X::Array{Float64}) = @ccall Clib.c_sum(length(X)::Csize_t, X::Ptr{Float64})::Float64

In [ ]:
c_sum(x)

In [ ]:
c_sum(x) ≈ sum(x)

#### Julia

In [ ]:
function jl_sum(A)
    s = zero(eltype(A)) # the correct zero type for A
    for a in A
        s += a
    end
    return s
end

In [ ]:
@btime c_sum($x) samples = 100 evals = 10;
@btime jl_sum($x) samples = 100 evals = 10;

### User type: Diagonal matrix

Let's create a simple custom `DiagonalMatrix` type that can represent square diagonal matrices, i.e.

$$ D = \left( \begin{matrix} x & 0 & 0 & 0 \\ 0 & y & 0 & 0 \\ 0 & 0 & z & 0 \\ 0 & 0 & 0 & \ddots \end{matrix} \right) $$

In [ ]:
struct DiagonalMatrix{T} <: AbstractArray{T,2}
    diag::Vector{T}
end

We integrate our `DiagonalMatrix` into Julia's type hierarchy by making it a subtype of `AbstractMatrix` to indicate **matrix-like behavior**. To actually make it behave like a matrix (a two-dimensional array) we implement at least (parts of) the [`AbstractArray` interface](https://docs.julialang.org/en/v1/manual/interfaces/#man-interface-array-1).

In [ ]:
# implement AbstractArray interface
function Base.getindex(D::DiagonalMatrix, i::Int, j::Int)
    if i == j
        return D.diag[i]
    else
        return zero(eltype(D))
    end
end

function Base.setindex!(D::DiagonalMatrix, v, i::Int, j::Int)
    if i == j
        D.diag[i] = v
    else
        throw(ArgumentError("cannot set off-diagonal entry ($i, $j)"))
    end
    return v
end

Base.size(D::DiagonalMatrix) = (length(D.diag), length(D.diag))

In [ ]:
D = DiagonalMatrix([1,2,3])

Note how it's automagically pretty printed (despite the fact that we never defined any special printing)!

But that's not it. Because of duck typing, all kinds of different functions now "just work".

In [ ]:
D + D # addition

In [ ]:
D * D # multiplication

In [ ]:
inv(D) # inversion

In [ ]:
using LinearAlgebra
eigen(D) # eigensolver

Of course, so far, these operations have suboptimal performance because they don't utilize the special structure inherent to our `DiagonalMatrix` but fall back to generic implementations. Let's implement an efficient addition for our diagonal matrix type.

In [ ]:
import Base: +

+(Da::DiagonalMatrix, Db::DiagonalMatrix) = DiagonalMatrix(Da.diag + Db.diag)

Let's compare our very rudamentary `DiagonalMatrix` against the standard `Diagonal` type that ships in the `LinearAlgebra` standard library.

In [ ]:
using BenchmarkTools
using LinearAlgebra

x = rand(1000);
Djl = Diagonal(x)
D = DiagonalMatrix(x)

@btime $Djl + $Djl;
@btime $D + $D;